# Script to compare data outputs

# Contents
[Setup](#Setup)

[Get the data](#Get-the-data)

[Data analysis](#Data-analysis)

[Save data](#Save-exp-data)

[Display analysis results](#Display-data-analysis-results)

[Work in progress](#Work-in-progress)

# Setup

##### Set up libaries, paths and data

In [ ]:
%matplotlib inline
import sys, os
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4
import operator
import IPython.display
import time
import glob
import pprint
sys.path.append('postdales')
import postdales as da

In [ ]:
# set paths
projectpath, exppath, workpath, ephemeral = da.tools.environment()
# define standard colors
light, dark, emphcolors = da.tools.colorschemes()
# define additional plot styles
linestyles = da.tools.plotstyles()
# define the plot style
# Can be configured in os.path.join(mpl._get_configdir(), 'stylelib').
# Default styles are in os.path.join(mpl.get_data_path(), 'stylelib').
mpl.style.use(dark)
# mpl.style.use(light)

# Get the data

[Back to Contents](#Contents)

[Display data analysis results](#Display-data-analysis-results)

[Work in progress](#Work-in-progress)

#### _Set the experiment numbers:_

In [ ]:
# expnrs = ['521', '522', '593', '574', '545', '556', '500']
expnrs = ['611', '602', '613', '624', '625', 
          '626', '610', '617', '618', '619']

## Load saved exp data object

In [ ]:
# set up list of exp class object
exps = []
for expnr in expnrs:
    exp = da.tools.load_saved_exp(expnr, 'figures/')
    exps.append(exp)

[Display data analysis results](#Display-data-analysis-results)

---
## Read in original data

##### Setup exp object and add data paths, specs, parameters and blocks

In [ ]:
# add data paths to exps
for exp in exps:
    exp.datapath = os.path.join(ephemeral, exp.name, '')
    exp.inpspath = os.path.join(exppath, exp.name, '')    
# add blocks and parameters to exps
for exp in exps:
    namoptions, blocksfile = da.tools.getinputs(exp.datapath, exp.inpspath)
    # get blocks
    blocks = da.tools.getblocks(blocksfile)
    exp.add_blockstats('blocks', blocks)
    # get parameters
    parameters = da.tools.getparameters(namoptions)
    exp.add_parameters(parameters)

##### Read in netcdf files

In [ ]:
for exp in exps:
    # get netcdf paths and fields
    # read from ephemeral first
    ncpaths = da.tools.getncpath(exp.datapath)
    # if ncpaths is empty, check workpath
    if not ncpaths:
        exp.datapath = os.path.join(workpath, exp.name, '')
        ncpaths = da.tools.getncpath(exp.datapath)
    # get fields
    ncfields = da.tools.getncdata(ncpaths)
    exp.add_data('ncfields', ncfields)
#     print("Available variables:\n")
#     for fdata in exp.ncfields:
#         print(fdata.title, ":\n", list(fdata.variables.keys()))

##### Select variables and read in data

In [ ]:
# dict of dict with selected variables
selectvars = {
             'fielddump' : {'xt' : 'xt', 'yt' : 'yt', 'zt' : 'zt', 
                            'xm' : 'xm', 'ym' : 'ym', 'zm' : 'zm',
                            'tfield' : 'time',
                            'u' : 'u', 'v' : 'v', 'w' : 'w',
                            'p' : 'pres', 'th' : 'thl'
                           },
             'xytdump' : {'zt' : 'zt', 'zm' : 'zm', 'tstats' : 'time', 
                          'ubar' : 'uxyt',  'vbar' : 'vxyt',  'wbar' : 'wxyt', 
                          'upwp' : 'upwpxyt', 'vpwp' : 'vpwpxyt', 'upvp' : 'upvpxyt',
                          'uw' : 'uwxyt', 'vw' : 'vwxyt', 'uv' : 'uvxyt',
                          'ww' : 'wwxyt',
                          'tke' : 'tketxyc',
                          'upup' : 'upuptxyc', 'vpvp' : 'vpvptxyc', 'wpwp' : 'wpwptxyc',
                          'usgs' :  'usgsxyt', 'vsgs' :  'vsgsxyt',
                          'pav' : 'pxyt'
                          , 'thl' : 'thlxyt', 'wpthl' : 'wpthlpxyt', 'wthl' : 'wthlxyt', 'thlpthlp' :  'thlpthlptxy', 'thlsgs' : 'thlsgsxyt'
                          , 'qtxyt' : 'qtxyt'
                         },
            'tdump' : {'xt' : 'xt', 'yt' : 'yt', 'zt' : 'zt', 
                       'xm' : 'xm', 'ym' : 'ym', 'zm' : 'zm',
                       'ut' : 'ut', 'vt' : 'vt', 'wt' : 'wt',
                       'upwpt' : 'upwpt', 'vpwpt' : 'vpwpt', 'upvpt' : 'upvpt',
                       'tket' : 'tketc',
                       'upupt' : 'upuptc', 'vpvpt' : 'vpvptc', 'wpwpt' : 'wpwptc',
                       'pt' : 'pt'
                       , 'thlt' : 'thlt', 'wpthlpt' : 'wpthlpt', 'thlpthlpt' : 'thlpthlpt'
                       , 'qtt' : 'qtt'
                      }
            }
for exp in exps:
    exp = da.tools.getncvars(exp, selectvars)

[Display data analysis results](#Display-data-analysis-results)

---
# Data analysis

[Back to Contents](#Contents)

[Display and save exp data](#Display-and-save-exp-data)

[Display data analysis results](#Display-data-analysis-results)

[Work in progress](#Work-in-progress)

### Colours and references

In [ ]:
# define references
referencelist, colorlist, markerlist = da.tools.referencelists()
# add colors and reference to exps
for counter, exp in enumerate(exps):
    exp.color = colorlist[counter]
    exp.marker = markerlist[counter]
    exp.reference = referencelist[counter]

## Basic data analysis

In [ ]:
for exp in exps:
#     # get parameters
#     parameters = da.tools.getparameters(namoptions)
#     exp.add_parameters(parameters)    
    # get dimensionalised blocks
    dimblocks = da.tools.getdimblocks(exp.blocks, exp.xm[:], exp.ym[:], exp.zm[:])
    exp.add_blockstats("dimblocks", dimblocks)
    # Block statistics
    a0 = exp.xsize * exp.ysize
    exp.add_blockstats("a0", a0)
    blockstats = da.stats.blockstats(dimblocks=exp.dimblocks, a0=exp.a0, zm=exp.zm[:])
    for name, value in blockstats.items():
        exp.add_blockstats(name, value)
    exp = da.stats.height_norm(exp)
    # get fluxes and gradients
    exp = da.stats.momentum_fluxes(exp)
    exp = da.stats.patch_momentum_fluxes(exp)  # set lowest entry to zero
    exp = da.stats.estimate_flux_gradients(exp)
    # get stresses
    exp = da.stats.estimate_pressure_gradient(exp)
    exp = da.stats.estimate_pressure_forcing(exp)
    exp = da.stats.estimate_drag(exp)
    # Friction velocity
    exp = da.stats.estimate_ustar(exp)
    # Flux norm
    exp = da.stats.flux_norm(exp)
# print what has been computed
for exp in exps:
    print(exp.reference, "Variables added:\n", list(exp.data.keys()), '\n')

---
## Further analysis
[Back to Contents](#Contents)

[Display and save exp data](#Display-and-save-exp-data)

[Display data analysis results](#Display-data-analysis-results)

### Compute block and canyon statistics

In [ ]:
for exp in exps:
#     # get parameters
#     parameters = da.tools.getparameters(namoptions)
#     exp.add_parameters(parameters)    
    # get dimensionalised blocks
    dimblocks = da.tools.getdimblocks(exp.blocks, exp.xm[:], exp.ym[:], exp.zm[:])
    exp.add_blockstats("dimblocks", dimblocks)
    # get street canyons
    dimcanyons, dimfullcanyons = da.tools.getcanyons(exp.dimblocks, [0, exp.xsize, 0, exp.ysize])
    exp.add_blockstats('dimcanyons', dimcanyons)
    exp.add_blockstats('dimfullcanyons', dimfullcanyons)
    a0 = exp.xsize * exp.ysize
    exp.add_blockstats("a0", a0)
    # Block statistics
    blockstats = da.stats.blockstats(dimblocks=exp.dimblocks, a0=exp.a0, zm=exp.zm[:], dimcanyons=exp.dimfullcanyons)
    for name, value in blockstats.items():
        exp.add_blockstats(name, value)
    # height norm
    exp = da.stats.height_norm(exp)

print("Computed street canyons.")
print("Computed block statistics.")
print("Computed height norms.")

### Compute vertical fluxes

In [ ]:
for exp in exps:
    # Get momentum fluxes
    varstart = 0
    varstop = None
    method = "last"  # "average", "last", "index"
    index = -1
    # get fluxes and gradients
    exp = da.stats.momentum_fluxes(exp)
    exp = da.stats.patch_momentum_fluxes(exp)  # set lowest entry to zero
print("Computed momentum fluxes.")

### Compute inflection point

In [ ]:
for exp in exps:
    # Estimate inflection point ze
    exp = da.stats.estimate_inflection_point(exp)
print("Estimated inflection point ze.")

### Compute forcing and drag

In [ ]:
for exp in exps:
    exp = da.stats.estimate_flux_gradients(exp)
    exp = da.stats.estimate_pressure_gradient(exp)
    exp = da.stats.estimate_pressure_forcing(exp)
    exp = da.stats.estimate_drag(exp)
    exp = da.stats.pressure_norm(exp)
    # Friction velocity
    exp = da.stats.estimate_ustar(exp)
    # Flux norm
    exp = da.stats.flux_norm(exp)
    # Centre of mass
    exp = da.stats.estimate_mass_centre(exp)
    # z_RSL and z_UCL
    exp = da.stats.estimate_layer_heights(exp)
print("Estimated total shear stress and drag.")
# define reference friction velocity of smooth wall case
for exp in exps:
    if '(g)' in exp.reference:
        expref = exp
exps = da.stats.ustar_norm(exps, expref)
print("Defined reference friction velocity for all experiments.")
# get a drag estimation polynomial
exps = da.stats.calculate_drag_polynomial_estimation(exps, degree=3)
print("Computed drag estimation polynomial.")

### Compute log law statistics

In [ ]:
for exp in exps:
    # z_RSL and z_ISL
#     exp = da.stats.estimate_layer_heights(exp)
    # add one type of blending height for log law estimate  fitting range
    zr_index = exp.zisl_index
    zr_index = 100
    exp.add_data("zr_index", zr_index)
#     fitting around estimated zr
    fitstart = exp.zr_index
    if fitstart < 15:
        fitstart = 50
    elif fitstart > exp.kmax - 50:
        fitstart = exp.kmax - 60
    fitstop = fitstart + 50
    # log law estimates
    exp = da.stats.estimate_displacement_length(exp, fitstart, fitstop, rangestop=45)
    exp = da.stats.estimate_roughness_length(exp, fitstart, fitstop)
    exp = da.stats.estimate_loglaw(exp)
print("Estimated displacement length zd.")
print("Estimated roughness length z0.")

---
# Save exp data

[Back to Contents](#Contents)

[Display data analysis results](#Display-data-analysis-results)

To get exp ready we need to delete or convert netcdf variables as they are not pickable.

In [ ]:
save = True
for exp in exps:
    if save is True:
        [exp.delete_data(d) for d in ['ncfields', 'ubar', 'upwp', 'uw']]
        # delete large netcdf variables, add data of small ones
        da.tools.delete_netcdfs(exp)
        with open(projectpath + '/src/jupyter/figures/exp-%s.pkl' %exp.name, 'wb') as file:
            da.tools.ExpStored.save(exp, file)
    print("\nExperiment data:")
    print(exp.fulllist())

---
# Display data analysis results

#### Block inputs
1. [Block layouts and blockfuntions](#Blocks-layouts-and-blockfunctions)  

#### Analysis of vertical profiles
1. [Vertical wind and flux profiles](#Vertical-wind-and-flux-profiles)
2. [Wind profile and momentum flux developments](#Wind-profile-and-momentum-flux-developments)
3. [Force balance and integrated force balance](#Force-balance-and-integrated-force-balance)

#### Compare between simulations

1. a) [Mean wind profiles](#Mean-wind-profiles)    
   b) [Normalised wind profiles in the urban canopy](#Normalised-wind-profiles-in-the-urban-canopy)   
2. [Total momentum fluxes](#Total-momentum-fluxes)    
3. [Drag](#Drag)  

#### Field plots
16. [Field plot](#Field-plot)
17. [Quiver and wind speed plot](#Quiver-and-wind-speed-plot)

[Save exp data](#Save-exp-data)

In [ ]:
mpl.style.use(dark)
# mpl.style.use(light)

[Back to Contents](#Contents)
## Blocks layouts and blockfunctions

In [ ]:
# setup
if len(exps) == 7:
    n = len(exps) - 1  # ignore reference solution
else:
    n = len(exps)
for exp in exps[:n]:
    # how often block layouts are duplicated
    divx = exp.xsize/120  # 4
    divy = exp.ysize/120  # 2
    nbl = int(len(exp.dimblocks)/(divx * divy))    
    limits=[0, exp.xsize/divx, 0, exp.ysize/divy, 0, exp.zsize/2]
    # plot
    fig = plt.figure(figsize=(12, 4))
    # geometry plot
    ax = fig.add_subplot(1, 4, 1, projection='3d')
    da.plot.blocks3d(exp.dimblocks[:nbl], ax=ax, 
                       facecolor=exp.color, edgecolor='k', 
                       limits=limits)
    plt.title("Blocks")
    ax.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
    ax.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
    ax.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
    ax.set_aspect('equal')
    ax.view_init(25, 235)
    ax.dist=12
    # layout plot
    ax = fig.add_subplot(1, 4, 2)
    da.plot.layout(exp.dimblocks[:nbl], ax=ax, limits=limits[:4], 
                     facecolor=exp.color)
    ax.set_xticks(np.linspace(*limits[0:2], 5))
    ax.set_yticks(np.linspace(*limits[2:4], 5))
    plt.title("Blocks plan layout")    
    # building density plot
    ax = fig.add_subplot(1, 4, 3)
    plt.plot(exp.blockfunction_lp, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=1, linestyle=":", color='grey')
    plt.xlabel("$\lambda_p(z)$")
    plt.ylabel("$z/z_H$")
    plt.title("Building area density")
    # building frontal density plot
    ax = fig.add_subplot(1, 4, 4)
    plt.plot(exp.blockfunction_lf, exp.zm*exp.normh, color=exp.color)
    plt.axhline(y=1, linestyle=":", color='grey')
    plt.xlabel("$\lambda_f(z)$")
    plt.ylabel("$z/z_H$")
    plt.title("Building frontal area density")

    plt.suptitle("%s" %exp.reference)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)
    plt.show()

In [ ]:
for exp in exps:
    print(exp.reference)
    print("lamda_p = ", exp.lp)
    print("lambda_f = ", exp.lf)
    print("no. of blocks: ", exp.nblocks)
    print("zmax = ", exp.zmax, "m")
    print("zh = ", exp.zh, "m")
    print("sigma_h = ", exp.sigma_h, "m")
#     print("Blocks:\n", exp.dimblocks)

[Back to Contents](#Contents)
## Vertical wind and flux profiles

In [ ]:
for exp in exps:
    # rounding time steps to nearest 10
    timestep = int(round(exp.t[-1]/10)*10)
    if len(exp.t) > 1:
        avtime = int(round((exp.t[-1] - exp.t[-2]) * 0.1) * 10)
        
    # plot figure
    fig = plt.figure(figsize=(3*2, 4*1))
    
    # mean wind
    ax = fig.add_subplot(1, 2, 1)
    plt.plot(exp.uav, exp.zt*exp.normh, color=exp.color)
    # z_H
    plt.scatter(exp.uav[exp.zh_tindex], exp.zh*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zh_tindex]-0.08, (exp.zh-8)*exp.normh, "$z_H$", 
             fontsize=8, color='grey')
    # z_max
    plt.scatter(exp.uav[exp.zmax_tindex], exp.zmax*exp.normh, marker='.', color='grey')
    plt.text(exp.uav[exp.zmax_tindex]-0.22, (exp.zmax+5)*exp.normh, "$z_{max}$", 
             fontsize=8, color='grey')

    plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
    plt.ylabel('Height $z/z_H$')
    plt.title("Mean $u$ wind profile")

    # fluxes
    ax = fig.add_subplot(1, 2, 2)
    plt.plot(exp.totflux, exp.zm*exp.normh, color=exp.color)
    plt.plot(exp.turbflux, exp.zm*exp.normh, label="turbulent flux", 
             linestyle=":", linewidth=1, color=exp.color)
    plt.plot(exp.dispflux, exp.zm*exp.normh, label='dispersive flux', 
             linestyle="--", linewidth=1, color=exp.color)
    plt.plot(exp.viscflux, exp.zm*exp.normh, label='viscous SGS flux', 
             linestyle="-.", linewidth=0.6, color=exp.color)

    # z_H
    plt.axhline(y=exp.zh*exp.normh, linewidth=0.5,  color="grey")
    plt.scatter((-0.05)*(1/exp.normf), exp.zh*exp.normh, marker='.', color='grey')
    plt.text((-0.08)*(1/exp.normf), (exp.zh-8)*exp.normh, "$z_H$", 
             fontsize=8, color='grey')
    # z_max
    plt.axhline(y=exp.zmax*exp.normh, linewidth=0.5, color='grey')
    plt.scatter((-0.05)*(1/exp.normf), exp.zmax*exp.normh, marker='.', color='grey')
    plt.text((-0.08)*(1/exp.normf), (exp.zmax+5)*exp.normh, "$z_{max}$", 
             fontsize=8, color='grey')

    plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
    plt.ylabel(r'Height $z/z_H$')
    plt.title("Vertical $u$ momentum flux")
    plt.legend(prop={'size': 'small'})

    if len(exp.t) > 1:
        plt.suptitle("%s Mean profiles after %s s with average time %s s" 
                     % (exp.name, timestep, avtime) )
    else:
        plt.suptitle("%s Mean profiles after %s s" % (exp.name, timestep) )
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()

[Back to Contents](#Contents)
## Wind profile and momentum flux developments

In [ ]:
# setup
varstart = -4
varstop = None
trange = slice(varstart, varstop)
# plot
for exp in exps:    
    try:
        times = exp.t[trange]
        varav = exp.uavs[trange]
        tots = exp.totfluxes[trange]
        turbs = exp.turbfluxes[trange]
        disps = exp.dispfluxes[trange]
    except Exception: # if slice out of bounds
        times = exp.t[:]
        varav = exp.uavs[:]  
        tots = exp.totfluxes[:]
        turbs = exp.turbfluxes[:]
        disps = exp.dispfluxes[:]  
        
    fig = plt.figure(figsize=(3*2, 4*1), dpi=250)
    # wind profile
    ax1 = fig.add_subplot(1, 2, 1)

    # get default text color black or white
    textcol = plt.xlabel('').get_color()
    if textcol is 'k':  # for dark background
        reverse=False
    else:
        reverse=True
    colorpal = sns.light_palette(exp.color, len(varav), reverse=reverse)

    for i, uav in enumerate(varav):
        # define variable length and average time
        if i - 1 < 0:
            try:
                t0 = int(exp.t[varstart - 1])
            except Exception:
                t0 = 0
        else:
            t0 = int(times[i-1])
        tstep = int(times[i])
        plt.plot(uav, exp.zt*exp.normh, label="%s s to %s s" %(t0, tstep),
                color=colorpal[i])
    plt.legend(fontsize='small', labelspacing=0.5)    
    plt.title("Wind profile development")
    plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
    plt.ylabel('Height $z/z_H$')

    # fluxes
    ax2 = fig.add_subplot(1, 2, 2)
    i = 0
    for tot, disp, turb in zip(tots, disps, turbs):
        # define variable length and average time
        if i - 1 < 0:
            try:
                t0 = int(exp.t[varstart - 1])
            except Exception:
                t0 = 0
        else:
            t0 = int(times[i-1])
        tstep = int(times[i])
        p = plt.plot(tot, exp.zm*exp.normh, label="%s s to %s s" %(t0, tstep),
                    color=colorpal[i])
        color = p[0].get_color()
        plt.plot(turb, exp.zm*exp.normh, linestyle=":", color=color)
        plt.plot(disp, exp.zm*exp.normh, linestyle="--", color=color)
        i += 1

#     plt.legend()
    plt.title("Flux development %s" % exp.name)
    plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
    plt.ylabel('Height $z/z_H$')
    
    plt.tight_layout()
    plt.show()

[Back to Contents](#Contents)
## Force balance and integrated force balance

In [ ]:
for exp in exps:
    # plot figure
    fig = plt.figure(figsize=(3*2, 4*1), dpi=250)
    
    # force balance
    ax1 = fig.add_subplot(1, 2, 1)
    # forcing    
    plt.plot(exp.forcing, exp.zm, 
             linestyle=linestyles[0], color=exp.color, label='_nolegend_')
#             , label=r"$F$")
    # drag
    plt.plot(exp.dragforce, exp.zm, 
             linestyle=linestyles[11], color=exp.color, label='_nolegend_') 
#              , label=r"$D$")
    # total flux
    plt.plot(exp.totflux_gradient, exp.zm, 
             linestyle=linestyles[2], color=exp.color, label='_nolegend_') 
#              , label=r"$\frac{\partial \tau}{\partial z}$")
    # turbulent flux
    plt.plot(exp.turbflux_gradient, exp.zm, 
             label= r"$-\frac{\partial}{\partial z} \langle \overline{w'u'} \rangle$",
             linewidth=1, linestyle=":", color='xkcd:light brown')
    # dispersive flux
    plt.plot(exp.dispflux_gradient, exp.zm, 
             label=r"$-\frac{\partial}{\partial z} \langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
    # SGS flux
    plt.plot(exp.viscflux_gradient, exp.zm, 
             label=r'$\frac{\partial}{\partial z} \nu_{SGS} \frac{\partial u}{\partial z}$',  
             linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    
    # forcing label
    plt.annotate("$F$", xy=(exp.forcing[-1], exp.zm[int(exp.zsize/2)]), xycoords='data', 
                 xytext=(10, 10), textcoords='offset points', color=exp.color)
    # drag label
    plt.annotate("$-D$", xy=(exp.dragforce[int(exp.zh_index - 1)], exp.zm[int(exp.zh_index)]), xycoords='data', 
                 xytext=(10, 10), textcoords='offset points', color=exp.color)
    # flux label
    plt.annotate(r"$\frac{\partial \tau}{\partial z}$", 
                 xy=(-exp.dragforce[int(exp.zh_index - 1)], exp.zm[int(exp.zh_index)]), xycoords='data', 
                 xytext=(-15, 10), textcoords='offset points', color=exp.color)

    # z_max
    plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
    if not exp.zmax == exp.zh:
        plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
                     xytext=(-21, 10), textcoords='offset points', color='grey')
    # z_h
    plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
    plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 0), textcoords='offset points', color='grey')


    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Forces and transport [m/s$^2$]")
    plt.ylabel('Height z [m]')
    plt.title("Forces")


    # fluxes
    ax2 = fig.add_subplot(1, 2, 2)
    # stress
    plt.plot(exp.totstress, exp.zm, 
             linestyle=linestyles[0], color=exp.color, label='_nolegend_')
    # drag
    plt.plot(exp.drag, exp.zm, 
             linestyle=linestyles[11], color=exp.color, label='_nolegend_') 
#              ,label=r"$\hat D$")
    # total flux
    plt.plot(exp.totflux, exp.zm, 
             linestyle=linestyles[2], color=exp.color, label='_nolegend_')
#              , label=r"$\tau$")
    # turbulent flux
    plt.plot(exp.turbflux, exp.zm, 
             label=r"$-\langle \overline{w'u'} \rangle$", 
             linewidth=1, linestyle=":", color='xkcd:light brown')
    
    # forcing label
    plt.annotate("$\hat F$", xy=(exp.totstress[3], exp.zm[int(exp.zmax_index/2)+3]), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)
    # drag label
    plt.annotate("$\hat D$", xy=(exp.drag[int(exp.zh_index + 1)], exp.zm[int(exp.zh_index + 5)]), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)
    # flux label
    plt.annotate(r"$\tau$", 
                 xy=(exp.turbflux[50], exp.zm[-1]/3), xycoords='data', 
                 xytext=(0, 0), textcoords='offset points', color=exp.color)

    # dispersive flux
    plt.plot(exp.dispflux, exp.zm, 
             label=r"$-\langle \bar{w}''\bar{u}'' \rangle$", 
             linewidth=1, linestyle="--", color='xkcd:greyish purple')
    # SGS flux
    plt.plot(exp.viscflux, exp.zm, 
             label=r'$\nu_{SGS} \frac{\partial \langle \bar{u} \rangle}{\partial z}$', 
             linestyle="-.", linewidth=0.6, color='xkcd:grey green')
    
    # z_max
    plt.axhline(y=exp.zmax, linewidth=0.5, color='grey')
    if not exp.zmax == exp.zh:
        plt.annotate(r"$z_{\max}$", xy=(0, exp.zmax/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 10), textcoords='offset points', color='grey')
    # z_h
    plt.axhline(y=exp.zh, linewidth=0.5,  color="grey")
    plt.annotate(r"$z_H$", xy=(0, exp.zh/exp.zsize), xycoords='axes fraction', 
                 xytext=(-21, 0), textcoords='offset points', color='grey')

    plt.legend(fontsize='small', labelspacing=0.5)
    plt.xlabel(r"Fluxes, drag, stress [m$^2$/s$^2$]")
    plt.ylabel('Height z [m]')
    plt.title("Integrated forces")
    
    plt.suptitle("%s" % exp.reference )
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()    
    

[Back to Contents](#Contents)
## Mean velocity profiles

In [ ]:
plt.figure()
i = 0
for exp in exps:
    plt.plot(exp.uav, exp.zt, label=exp.reference, color=exp.color, marker=exp.marker, markevery=100 + i)
    i += 10
plt.legend()
plt.xlabel(r"Wind speed $\langle \bar u \rangle(z)$ [m/s]")
plt.ylabel('Height z [m]')
plt.title("Mean wind profiles")
plt.show()
for exp in exps:
    dzs = np.gradient(exp.zt)
    print(exp.reference, "height average:", np.sum(exp.uav*dzs)/exp.zsize)


[Back to Contents](#Contents)
## Total momentum fluxes

In [ ]:
#total fluxes CSA
plt.figure()
i = 0
for exp in exps:
    i += 5
    plt.plot(exp.totflux, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[1, 100 + i])
    plt.plot(exp.totstress[:exp.zmax_index+3]*normu, exp.zm[:exp.zmax_index+3], 
             color=exp.color, linestyle="--", linewidth=0.5,
             marker=exp.marker, markevery=[1])
plt.legend()
plt.title("Total vertical momentum fluxes")
plt.xlabel(r"Flux $\tau(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

[Back to Contents](#Contents)
## Drag

In [ ]:
plt.figure()
i = 0
for exp in exps:
    i += 5
    plt.plot(exp.drag, exp.zm, label=exp.reference, color=exp.color, marker=exp.marker, markevery=[0])
    plt.plot(exp.totstress, exp.zm, label='_nolegend_', color=exp.color, linestyle="--", linewidth=0.5, marker=exp.marker, markevery=[1])
plt.legend()
plt.title("Drag")
plt.xlabel("Integrated drag $\hat D(z)$ [m$^2$/s$^2$]")
plt.ylabel('Height z [m]')
plt.show()

[Back to Contents](#Contents)
## Vertical profiles

##### Print available variables

In [ ]:
for exp in exps:
    # all available variables
    ncvariables = []  # list with nc variables
    ncvarnames = []  # list with names of variables in exp.data
    for name, variable in exp.data.items():
        if isinstance(variable, (netCDF4._netCDF4.Variable)):
            ncvariables.append(variable)
            ncvarnames.append(name)
    
    # print only 1D vertical profile variables:
    verticalaxes = ['zt', 'zm']
    ncfields1D = []
    ncfields1Dnames = []
#     for variable in ncvariables:
    for name in ncvarnames:
        variable = exp.data[name]
        if any(axis in verticalaxes for axis in variable.dimensions):
            if variable.ndim < 3:  # only time and z dimensions
                if not variable.name in verticalaxes:  # do not print axis variables themselves
                    ncfields1D.append(variable)
                    ncfields1Dnames.append(name)
    print(exp.name, "Available 1D  vertical profile variables:\n", sorted(ncfields1Dnames), "\n")

### Any 1D quantity

In [ ]:
var = 'wpwp'
zaxis = exp.data[var].dimensions[1]

for exp in exps:
    plt.figure()
    plt.plot(exp.data[var][-1, :], exp.data[zaxis], label=exp.reference, color=exp.color, marker=exp.marker, markevery=[1])
    plt.legend()
    plt.ylabel(zaxis)
    plt.xlabel(var)
    plt.show()

[Back to Contents](#Contents)
## Field plots

#### Print available fields for field plots

In [ ]:
for exp in exps:
    # all available variables
    ncvarnames = []  # list with names of variables in exp.data
    for name, variable in exp.data.items():
        if isinstance(variable, (netCDF4._netCDF4.Variable)):
            ncvarnames.append(name)    
            
    # print only 3D field variables:
    fieldaxes = ['xt', 'xm', 'yt', 'ym']
    ncfields3D = []
    ncfields3Dnames = []
    for name in ncvarnames:
        variable = exp.data[name]
        if any(axis in fieldaxes for axis in variable.dimensions):
            if not variable.name in fieldaxes:  # do not print axis variables themselves
                ncfields3D.append(variable)
                ncfields3Dnames.append(name)
    print(exp.name, "Available 2D or 3D fields:\n", sorted(ncfields3Dnames), "\n")
#     print(exp.name, "Available 2D or 3D fields:\n", ncfields3D, "\n")

### Field plot

In [ ]:
# define field variable
varname = 'u'
# define positions for field plot
tpos = -1
spos = 25
# define fixed axis for field plot
fix = 'yaxis'
# define plot limits
# limits=[0, 60, 0, 40]
limits=None
# define a colour map
cbarmap = plt.cm.get_cmap('RdYlBu_r')

for exp in exps:   
    field = da.plot.prepare_fielddata(exp, varname, 
                                      fix, spos, tpos, 
                                      limits)

    field = da.plot.prepare_contourplot(field, cbarmap, 
                                        cbartype='centered')
#     field.block_opts = {'facecolor' : "none", 'edgecolor' : 'k'}
    field.block_opts = {'facecolor' : "w", 'edgecolor' : 'w'}
#     plt.figure()
#     da.plot.contourfield(field)
#     plt.tight_layout()
#     plt.show()
    field = da.plot.prepare_layoutplot(field)
    da.plot.layout_fieldplot(field)

[Back to Contents](#Contents)
### Quiver and wind speed plot

In [ ]:
# define wind variables
var1 = 'u'
var2 = 'v'
var3 = 'w'
# define positions for field plot
tpos = -1
spos = 25
# define fixed axis for field plot
fix = 'yaxis'
# define plot limits
# limits=[0, 100, 0, 100]
limits=None
sparse=5
# define colour map
# cbarmap = plt.cm.get_cmap('YlOrRd')
cbarmap = plt.cm.get_cmap('RdYlBu_r')

for exp in exps:    
    field = da.plot.prepare_quiverplot(exp, var1, var2, var3, fix,
                               spos, tpos, limits, sparse)
    
    field = da.plot.prepare_contourplot(field, cbarmap)  
    
    field = da.plot.prepare_layoutplot(field)

    da.plot.layout_fieldplot(field, quiver=True)

[Back to Contents](#Contents)